In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import ast 
import plotly.express as px
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import matplotlib.ticker as mtick
import seaborn as sns
# plt.style.use('fivethirtyeight')
# plt.rcParams['figure.figsize'] = 10,6
# plt.xkcd() 
%matplotlib inline

In [ ]:
import os
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from wordcloud import WordCloud
from geopy.geocoders import Nominatim
from folium.plugins import HeatMap
import folium
from tqdm import tqdm
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.colors as mcolors
from sklearn.manifold import TSNE
from gensim.models import word2vec
import nltk

In [ ]:
df = pd.read_csv('/kaggle/input/zomato-bangalore-restaurants/zomato.csv')

In [ ]:
print('Number of Rows=',df.shape[0],' Number of Columns = ',df.shape[1])
df.head()

In [ ]:
df.info()

In [ ]:
data = df.copy()

In [ ]:
column_to_drop = ['address','url', 'phone']
data.drop(columns=column_to_drop, axis=1,inplace=True)

In [ ]:
data.rename(columns={'approx_cost(for two people)': 'average_cost', 'listed_in(type)': 'meal_type'}, inplace=True)

In [ ]:
# check for duplicate values
duplicate_data = data[data.duplicated() == True]

In [ ]:
# print('Number of duplicate entries = ',len(duplicate_data.name))
data.drop_duplicates(inplace=True)

In [ ]:
# check for null values what percentage of data is null 
(((data.isna().sum()/data.shape[0])*100).round(2))

We can see that df.dropna would remove more than 50% of the data hence we consider the rate column 

In [ ]:
data.rate.unique()

In [ ]:
# convert "NEW" & "-" to np.nan
data.rate.replace(('NEW','-'),np.nan,inplace =True)
# first make it as string
data.rate = data.rate.astype('str')
# remove the "/5" 
data.rate = data.rate.apply(lambda x: x.replace('/5','').strip())
# convert column type to float
data.rate = data.rate.astype('float')

In [ ]:
data["average_cost"] = data["average_cost"].str.replace(",","").astype(float)

In [ ]:
corr = data.corr()
ax = sns.heatmap(
    corr, 
    vmin=0, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
to=df.groupby(["listed_in(type)","name"]).sum()

lis=list(to.index)

val1=[]
for i in range(0,len(lis)):
    val1.append(lis[i][0])


st=set(val1)

type_rest=[]
cnt=[]
for i in st:
    type_rest.append(i)
    cnt.append(val1.count(i))
    
    

type_rest,cnt
dff=pd.DataFrame()
dff["Types of Zomato Restaurants in Bangalore"]=type_rest
dff["Count of Restaurants in Bangalore"]=cnt
fig = px.bar(dff, x="Types of Zomato Restaurants in Bangalore", y="Count of Restaurants in Bangalore",
         height=400)
fig.show()


In [ ]:
len(df[df.menu_item == '[]'])

In [ ]:
39617/51517

In [ ]:
data.corr()

In [ ]:
sns.boxplot(data=data)

In [ ]:
plt.figure(figsize=(10,7))
chains=data['name'].value_counts()[:20]
sns.barplot(x=chains,y=chains.index,palette='deep')
plt.title("Most famous restaurants chains in Bangaluru")
plt.xlabel("Number of outlets")

In [ ]:
# check for duplicate values
data2 = data.copy()
duplicate_data = data2[data2.duplicated() == True]
print('Number of duplicate entries = ',len(duplicate_data.name))
data.drop_duplicates(inplace=True)

In [ ]:
plt.figure(figsize=(6,5))
# rating=df['rate'].dropna().apply(lambda x : float(x.split('/')[0]) if (len(x)>3)  else np.nan ).dropna()
rating = data['rate']
sns.distplot(rating,bins=20)

1.Almost more than 50 percent of restaurants has rating between 3 and 4. <br>
2.Restaurants having rating more than 4.5 are very rare.

In [ ]:
plt.figure(figsize=(10,7))
sns.scatterplot(x="rate",y='average_cost',hue='online_order',data=data)
plt.show()

There is not much noticeable correlation between the rating and average cost wrt availability of online order .

In [ ]:
plt.figure(figsize=(6,6))
sns.distplot(data['average_cost'])
plt.show()

We can see that the distribution if left skewed. <br>
This means almost 90percent of restaurants serve food for budget less than 1000 INR

In [ ]:
plt.figure(figsize=(7,7))
rest=data['rest_type'].value_counts()[:20]
sns.barplot(rest,rest.index,palette="rocket")
plt.title("Restaurant types")
plt.xlabel("count")

<h3>Most common types of restaurants in Bangalore</h3> <br>
    We can see that quick bytes type of restaurant dominates!!

In [ ]:
plt.figure(figsize=(7,7))
Rest_locations=data['location'].value_counts()[:20]
sns.barplot(Rest_locations,Rest_locations.index,palette="rocket")

<h3>Restaurant distribution in Bangalore</h3>
We can see that BTM,HSR and Koranmangala 5th block has the most number of restaurants.

In [ ]:
plt.figure(figsize=(7,7))
cuisines=data['cuisines'].value_counts()[:10]
sns.barplot(cuisines,cuisines.index,palette="rocket")
plt.xlabel('Count')
plt.title("Most popular cuisines of Bangalore")